### Convert MAESTRO data to financial time bars

MAESTRO is Google Magenta's dataset of 200+ hours of annotated piano performance MIDI data.

In [1]:
from __future__ import division
from IPython.display import display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as scp
import magenta
import os
%matplotlib inline

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

PLOT_WIDTH = 1200
PLOT_HEIGHT = 800

def hheader(x):
    print("#########################################")
    print("### {}".format(x))
    print("#########################################")

# Magenta dependencies:
# https://github.com/magenta/magenta

# Magenta uses pretty_midi to deal with midi files
import pretty_midi

C:\Users\echow\Miniconda3\envs\tensorflow\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
C:\Users\echow\Miniconda3\envs\tensorflow\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
C:\Users\echow\Miniconda3\envs\tensorflow\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Could

In [2]:
INPUT_PATH = "data/maestro/maestro-v3.0.0/"
add_input_path = lambda x: "{}/{}".format(INPUT_PATH, x).replace("//", "/")
OUTPUT_PATH = "data_processed/maestro/"
add_output_path = lambda x: "{}/{}".format(OUTPUT_PATH, x).replace("//", "/")
if not os.path.exists(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)

### Read in MIDI metadata and pieces

In [3]:
### read in MIDI metadata
midi_metadata = pd.read_csv(add_input_path("maestro-v3.0.0.csv"))
midi_metadata.head()

,canonical_composer,canonical_title,split,year,midi_filename,audio_filename,duration
0,Alban Berg,Sonata Op. 1,train,2018,2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R...,2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R...,698.661160
1,Alban Berg,Sonata Op. 1,train,2008,2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MI...,2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MI...,759.518471
2,Alban Berg,Sonata Op. 1,train,2017,2017/MIDI-Unprocessed_066_PIANO066_MID--AUDIO-...,2017/MIDI-Unprocessed_066_PIANO066_MID--AUDIO-...,464.649433
3,Alexander Scriabin,"24 Preludes Op. 11, No. 13-24",train,2004,2004/MIDI-Unprocessed_XP_21_R1_2004_01_ORIG_MI...,2004/MIDI-Unprocessed_XP_21_R1_2004_01_ORIG_MI...,872.640588
4,Alexander Scriabin,"3 Etudes, Op. 65",validation,2006,2006/MIDI-Unprocessed_17_R1_2006_01-06_ORIG_MI...,2006/MIDI-Unprocessed_17_R1_2006_01-06_ORIG_MI...,397.857508


In [4]:
import os

midi_files = []
for root, dirs, files in os.walk(INPUT_PATH):
    for file in files:
        if file.endswith(".midi"):
            print(os.path.join(root, file))
            midi_files.append(os.path.join(root, file))

data/maestro/maestro-v3.0.0/2004\MIDI-Unprocessed_SMF_02_R1_2004_01-05_ORIG_MID--AUDIO_02_R1_2004_05_Track05_wav.midi
data/maestro/maestro-v3.0.0/2004\MIDI-Unprocessed_SMF_02_R1_2004_01-05_ORIG_MID--AUDIO_02_R1_2004_06_Track06_wav.midi
data/maestro/maestro-v3.0.0/2004\MIDI-Unprocessed_SMF_02_R1_2004_01-05_ORIG_MID--AUDIO_02_R1_2004_08_Track08_wav.midi
data/maestro/maestro-v3.0.0/2004\MIDI-Unprocessed_SMF_02_R1_2004_01-05_ORIG_MID--AUDIO_02_R1_2004_10_Track10_wav.midi
data/maestro/maestro-v3.0.0/2004\MIDI-Unprocessed_SMF_05_R1_2004_01_ORIG_MID--AUDIO_05_R1_2004_02_Track02_wav.midi
data/maestro/maestro-v3.0.0/2004\MIDI-Unprocessed_SMF_05_R1_2004_01_ORIG_MID--AUDIO_05_R1_2004_03_Track03_wav.midi
data/maestro/maestro-v3.0.0/2004\MIDI-Unprocessed_SMF_05_R1_2004_02-03_ORIG_MID--AUDIO_05_R1_2004_06_Track06_wav.midi
data/maestro/maestro-v3.0.0/2004\MIDI-Unprocessed_SMF_07_R1_2004_01_ORIG_MID--AUDIO_07_R1_2004_02_Track02_wav.midi
data/maestro/maestro-v3.0.0/2004\MIDI-Unprocessed_SMF_07_R1_2004_

### Process pieces by converting into time bars

In [47]:
"""
Process pieces
"""

all_dfs = []
for t in range(len(midi_files)):
    curr_file = midi_files[t]
    print("Processing file {} / {} ...".format(t+1, len(midi_files)))
    
    midi_test = pretty_midi.PrettyMIDI(curr_file)
    solo_piano_part = midi_test.instruments[0]
    df_notes = pd.DataFrame([(n.start, n.end, n.pitch, n.velocity, n.duration) for n in solo_piano_part.notes],
                            columns=['start', 'end', 'pitch', 'velocity', 'duration'])
    ### get metadata
    df_metadata = midi_metadata[midi_metadata['midi_filename']==curr_file.replace(INPUT_PATH, "").replace("\\", "/")]
    df_metadata = df_metadata.rename(columns={"duration": "total_duration"})
    df_metadata.drop(columns="audio_filename", inplace=True) # don't need audio for now
    curr_filename = df_metadata['midi_filename'].values[0].replace("/", "__")
    
    ### combine (so have metadata features for each row)
    df_curr = pd.merge(df_notes, df_metadata, how="cross")
    
    ### need to make sure sorted in order of notes (start)
    df_curr = df_curr.sort_values(by='start')
    
    """ Process each piece """
    
    ### Make sure everything starts at t=0
    ### TODO(echow): will have to update the audio files to also mention this
    start_time = float(df_curr['start'].head(1))
    df_curr['start_offset_before_first_note'] = start_time
    df_curr['start'] = df_curr['start'].apply(lambda x: x - start_time)
    df_curr['end'] = df_curr['end'].apply(lambda x: x - start_time)
    df_curr['total_duration'] = df_curr['total_duration'].apply(lambda x: x - start_time)
    
    """ We have to do some sampling here, because otherwise it is difficult to construct good features.
        So, the sampling here will coincide with the music being played in real-time.
    """
    
    """ Version 1: time bars
        (easiest to play back in real-time)
    """
    EVERY_N_SEC = 1
    df_curr_time = df_curr.copy()
    df_curr_time['start_sec'] =  (df_curr_time['start'] / EVERY_N_SEC).apply(np.floor).astype(int) * EVERY_N_SEC
    # round to N digits
    tmp = df_curr_time.select_dtypes(include=[np.number])
    df_curr_time.loc[:, tmp.columns] = np.round(tmp, decimals=5)
    # add filename
    df_curr_time['curr_filename'] = curr_filename
    
    ### aggregate by sample
    ### this is different from the more advanced features we will create - this is only for working
    ### with the raw music data at a usable (sampled) level
    def str_concat(x): return(','.join([str(s) for s in x]))
    df_curr_time_agg = df_curr_time.groupby(['start_sec'], as_index=False).agg({
        'start': [np.min, np.mean, np.median, np.max, str_concat],
        'end': [np.min, np.mean, np.median, np.max, str_concat],
        'pitch': [np.min, np.mean, np.median, np.max, str_concat],
        'velocity': [np.min, np.mean, np.median, np.max, str_concat],
        'duration': [np.min, np.mean, np.median, np.max, str_concat],
        # metadata features - same for all observations
        'canonical_composer': [pd.Series.mode],
        'split': [pd.Series.mode],
        'year': [pd.Series.mode],
        'total_duration': [pd.Series.mode],
        'curr_filename': [pd.Series.mode] })
    df_curr_time_agg.columns = ['_'.join([cc for cc in c if len(cc) > 0]).replace("amin", "min").replace("amax", "max")
                     for c in list(df_curr_time_agg.columns)]
    
    ### add missing time bars (just for consistency time series analysis - doesn't affect quality of data)
    df_curr_time_agg = pd.merge(df_curr_time_agg,
                             pd.DataFrame(range(0, df_curr_time_agg.shape[0], 1), columns=["start_sec"]), on="start_sec",
                            how='right').sort_values(by="start_sec")
    
    ### save
    curr_fp = curr_filename.replace(".midi", ".csv")
    df_curr_time_agg.to_csv(add_output_path(curr_fp))
    all_dfs.append(df_curr_time_agg)
    
    """ Version 2: information bars
        Skip this and volume bars for now - hard to play back in real-time.
    """
    
    
    """ Version 3: volume bars
        Skip this and volume bars for now - hard to play back in real-time.
    """
    
    
    print("... converted!")

Processing file 1 / 1276 ...


Exception: 

In [ ]:
# TODO: rerun with the above code to fill in missing start_sec bars

In [22]:
""" Also create a big music stream from all those MIDI files concatenated,
    representing one continuous stream of music
"""

df_all_time_agg = pd.concat([df.assign(piece="piece_{}".format(dx)) for dx, df in enumerate(all_dfs)])
columns = df_all_time_agg.columns.tolist()[-1:] + df_all_time_agg.columns.tolist()[:-1]
df_all_time_agg = df_all_time_agg[columns]

### preview
display(df_all_time_agg.head())
print(df_all_time_agg.shape)

### save
all_fp = "maestro_full_music_stream.csv"
df_all_time_agg.to_csv(add_output_path(all_fp))
print("Wrote to: {}".format(add_output_path(all_fp)))

# df_all_time_agg.groupby(['piece']).size().sort_values()

,piece,start_sec,start_min,start_mean,start_median,start_max,start_str_concat,end_min,end_mean,end_median,end_max,end_str_concat,pitch_min,pitch_mean,pitch_median,pitch_max,pitch_str_concat,velocity_min,velocity_mean,velocity_median,velocity_max,velocity_str_concat,duration_min,duration_mean,duration_median,duration_max,duration_str_concat,canonical_composer_mode,split_mode,year_mode,total_duration_mode,curr_filename_mode
0,piece_0,0,0.00000,0.542394,0.617185,0.94479,"0.0,0.18646,0.19583,0.37083,0.54062,0.69375,0....",0.09687,0.749271,0.718230,1.43021,"0.09687,0.40417,0.70104,0.53854,0.66042,0.7354...",55,66.000000,69.0,74,"71,55,71,59,62,72,67,74,57,72",44,60.300000,58.0,77,"60,44,54,55,52,76,56,68,61,77",0.04167,0.206875,0.143750,0.53958,"0.09687,0.21771,0.50521,0.16771,0.11979,0.0416...",Johann Sebastian Bach,train,2004,967.16405,2004__MIDI-Unprocessed_SMF_02_R1_2004_01-05_OR...
1,piece_0,1,1.00521,1.432576,1.465620,1.98750,"1.00521,1.05625,1.07917,1.23542,1.43021,1.4656...",1.08958,1.704828,1.581250,2.35521,"1.08958,1.42708,1.1875,1.41667,1.49687,1.67083...",59,67.909091,67.0,74,"74,72,67,66,71,64,72,74,59,62,66",35,56.363636,58.0,68,"51,60,57,58,68,35,47,68,50,63,63",0.06667,0.272254,0.192710,0.76875,"0.08438,0.37083,0.10833,0.18125,0.06667,0.2052...",Johann Sebastian Bach,train,2004,967.16405,2004__MIDI-Unprocessed_SMF_02_R1_2004_01-05_OR...
2,piece_0,2,2.15729,2.560937,2.616665,2.90000,"2.15729,2.36042,2.52708,2.70625,2.71458,2.9",2.50938,3.062153,2.903645,4.23542,"4.23542,2.50938,2.71354,2.93333,2.87396,3.10729",59,71.333333,72.5,79,"67,71,74,78,59,79",58,68.333333,69.5,78,"58,66,78,77,58,73",0.14896,0.501215,0.196875,2.07812,"2.07812,0.14896,0.18646,0.22708,0.15938,0.20729",Johann Sebastian Bach,train,2004,967.16405,2004__MIDI-Unprocessed_SMF_02_R1_2004_01-05_OR...
3,piece_0,3,3.08021,3.500116,3.431250,3.98542,"3.08021,3.08958,3.25625,3.42604,3.43125,3.6041...",3.24062,3.698147,3.612500,4.21458,"3.25,3.24062,3.42812,3.59167,3.6125,3.80521,3....",57,69.666667,72.0,79,"76,60,79,74,59,79,72,57,71",47,64.333333,68.0,76,"76,54,68,70,59,63,70,47,72",0.15104,0.198033,0.173960,0.38750,"0.16979,0.15104,0.17188,0.16563,0.18125,0.2010...",Johann Sebastian Bach,train,2004,967.16405,2004__MIDI-Unprocessed_SMF_02_R1_2004_01-05_OR...
4,piece_0,4,4.17917,4.543751,4.541145,4.92396,"4.17917,4.18542,4.34792,4.54062,4.54167,4.7177...",4.35833,4.792836,4.805730,5.26562,"4.35833,4.52187,4.54271,4.70417,5.26562,4.9072...",55,68.125000,70.0,81,"72,66,81,72,55,71,69,59",61,67.750000,68.5,73,"73,61,73,70,67,70,66,62",0.04688,0.249089,0.184375,0.72396,"0.17917,0.33646,0.19479,0.16354,0.72396,0.1895...",Johann Sebastian Bach,train,2004,967.16405,2004__MIDI-Unprocessed_SMF_02_R1_2004_01-05_OR...


(671603, 32)
Wrote to: data_processed/maestro/maestro_full_music_stream.csv
